<a href="https://colab.research.google.com/github/nbacomdados/nbacomdados/blob/main/notebooks/leitura_jogos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import json

In [2]:
def retorna_header():
  headers: dict = {'Accept': '*/*',
                  'Accept-Encoding': 'gzip, deflate, br',
                  'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
                  'Connection': 'keep-alive',
                  'Host': 'stats.nba.com',
                  'Origin': 'https://www.nba.com',
                  'Referer': 'https://www.nba.com/',
                  'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
                  'sec-ch-ua-mobile': '?0',
                  'Sec-Fetch-Dest': 'empty',
                  'Sec-Fetch-Mode': 'cors',
                  'Sec-Fetch-Site': 'same-site',
                  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

  return headers                

In [74]:
GameID = "0042000217" #@param {type:"string"}

dict_df = {}
status_code = 200
df_players_all = pd.DataFrame()
df_game_statistics_all = pd.DataFrame()
Q = 1

while status_code != None:

	startPeriod = Q
	endPeriod = Q

	num_Q = f'Q{startPeriod}' if startPeriod == endPeriod else f'T{startPeriod}-{endPeriod}'

	rangeType = '1'
	startRange = '0'
	endRange = '31800'

	headers = retorna_header()  
	url_base = f'https://stats.nba.com/stats/boxscoretraditionalv3?GameID={GameID}&LeagueID=00&'
	url_stats = f'{url_base}endPeriod={endPeriod}&endRange={endRange}&rangeType={rangeType}&startPeriod={startPeriod}&startRange={startRange}'

	with requests.Session() as sessao:
			resposta: requests.models.Response = sessao.get(url_stats, headers=headers)

	y = resposta.json()
	
	status_code = y['boxScoreTraditional']['homeTeam']['teamName']

	if status_code != None:

		name_homeTeam = y['boxScoreTraditional']['homeTeam']['teamName']
		name_awayTeam = y['boxScoreTraditional']['awayTeam']['teamName']
		gameId = y['boxScoreTraditional']['gameId']

		df_players_homeTeam = pd.json_normalize(y['boxScoreTraditional']['homeTeam']['players'])
		df_players_homeTeam['teamName'] = name_homeTeam
		df_players_awayTeam = pd.json_normalize(y['boxScoreTraditional']['awayTeam']['players'])
		df_players_awayTeam['teamName'] = name_awayTeam

		df_players = df_players_awayTeam.append(df_players_homeTeam)
		df_players['Q'] = Q
		df_players_all = df_players_all.append(df_players)

		df_awayTeam_statistics = pd.json_normalize(y['boxScoreTraditional']['awayTeam']['statistics'])
		df_homeTeam_statistics = pd.json_normalize(y['boxScoreTraditional']['homeTeam']['statistics'])
		df_game_statistics = df_homeTeam_statistics.append(df_awayTeam_statistics)
		df_game_statistics.index = [name_homeTeam, name_awayTeam]
		
		df_game_statistics['Q'] = Q
		
		df_game_statistics_all = df_game_statistics_all.append(df_game_statistics)

	Q += 1